In [1]:
import dask
from dask.distributed import Client, progress
from dask import compute, delayed
from dask_cloudprovider.aws import FargateCluster

In [2]:
my_vpc = 'vpc-bd20e6c4'# your vpc
my_subnets =['subnet-0457554c'] # your subnets
cpu = 2 
ram = 8

In [3]:
cluster = FargateCluster(
    n_workers=1,
    image='509798658644.dkr.ecr.eu-west-1.amazonaws.com/dask:latest',
    scheduler_mem=8192,
    vpc=my_vpc,
    subnets=my_subnets,
    worker_cpu=int(cpu * 1024),
    worker_mem=int(ram * 1024),
    cloudwatch_logs_group="fargate_dask_log_group",
    task_role_policies=['arn:aws:iam::aws:policy/AmazonS3FullAccess'],
    scheduler_timeout='20 minutes')

/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/amygda_dask/lib/python3.9/contextlib.py:126: UserWarning: Creating your cluster is taking a surprisingly long time. This is likely due to pending resources on AWS. Hang tight! 
  next(self.gen)


In [4]:
cluster.adapt(minimum=1,
              maximum=100)

2022-04-28 13:23:03,953 - distributed.deploy.adaptive - INFO - Adaptive scaling started: minimum=1 maximum=100


In [5]:
client = Client(cluster)
client

<Client: 'tcp://172.31.17.74:8786' processes=1 threads=2, memory=7.45 GiB>

In [7]:
import s3fs
import dask.dataframe as dd

df = dd.read_csv(
    's3://nyc-tlc/trip data/yellow_tripdata_2018-*.csv',  
    parse_dates=['tpep_pickup_datetime','tpep_dropoff_datetime'],
    assume_missing=True
)

In [8]:
len(df)

102804250